In [2]:
# Import libraries and packages for the project 

from selenium import webdriver
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import random
import numpy as np
import pandas as pd
from underthesea import sentiment


print('- Finish importing packages')

- Finish importing packages


In [3]:
raw = pd.read_csv('ShopeeMall.csv')
raw.head()

,Order,Name,URL
0,1,Abbott,https://shopee.vn/abbottofficial
1,2,Adidas Official Store,https://shopee.vn/adidasofficialstore
2,3,Alobuy,https://shopee.vn/1khovn_bach_hoa_tien_loi
3,4,Amak,https://shopee.vn/nhasachamak
4,5,An Nguyên,https://shopee.vn/sachhayquanhieu


In [5]:
df = np.array(raw)

In [5]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver,20)
url = "https://shopee.vn/"
# Login to shopee account
login_url = "https://shopee.vn/buyer/login/qr?next=https%3A%2F%2Fshopee.vn%2Fuser%2Fpurchase%2F"
driver.get(login_url)
sleep(40)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [/home/dijkstra/.wdm/drivers/chromedriver/linux64/104.0.5112.20/chromedriver] found in cache


In [6]:

headers = ['Order', 'Name', 'URL', "Negative","Positive"]
update_data = []

count = 0
negative = 0
positive = 0
total_cmt = 0
tmp_neg = 0
tmp_pos = 0

for shop in tqdm(df,desc="Processing: "):
    product_list_url = []
    count = 0
    negative = 0
    positive = 0
    try:
        my_url = shop[2]+"?page=0&sortBy=sales"
        driver.get(my_url)
        print('- Accessing shop: ', shop[2])
        sleep(2)
        
    except:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue

    print('-- Getting data....')

    for i in range(4):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
        sleep(0.3)
        loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
        sleep(0.3)
        old_page_source = driver.page_source

        items = loop_page_source.find('div',class_="shop-search-result-view").find_all('a',href=True)

        product_list_url.append(items)
        wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click()
        sleep(0.3)
        new_page_source = driver.page_source
        if new_page_source == old_page_source:
            break
        sleep(2)
    print('-- Finished get data')

    print("--- Access product pages")
    print("************************")
    for page in product_list_url:
        for products in tqdm(page,desc="Current/Total"):
            
            total_cmt = 0
            tmp_neg = 0
            tmp_pos = 0

            count+=1
            if(count==101):
                break
            product_url = url + products.get('href')
            
            driver.get(product_url)
            sleep(2)

            while(1):
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
                sleep(0.1)
                product_detail_page_source = BeautifulSoup(driver.page_source,"html.parser")
                sleep(0.2)
                old_page_source = driver.page_source

                comments = product_detail_page_source.find_all('div', class_="Em3Qhp") # Get comment
                
                if(len(comments)==0): # Section have no comments
                    if total_cmt==0: # First comment section has no comment
                        sleep(2)
                    break

                for cmt in comments:
                    my_text = cmt.text.strip()
                    if(sentiment(my_text)=="negative"):
                        tmp_neg+=1
                    else:
                        tmp_pos+=1

                total_cmt = tmp_neg+tmp_pos

                try:
                    wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"shopee-icon-button--right"))).click()
                except TimeoutException:
                    continue
                sleep(0.3)
                new_page_source = driver.page_source
                if new_page_source == old_page_source or total_cmt>=200:
                    negative+= tmp_neg
                    positive+= tmp_pos
                    break
    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:negative,headers[4]:positive})
    print("************************")

print("Finished crawling data!")


Processing:   0%|          | 0/26 [00:00<?, ?it/s]

- Accessing shop:  https://shopee.vn/chaangvietnam
-- Getting data....
-- Finished get data
--- Access product pages
************************


Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

...

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

Current/Total:   0%|          | 0/30 [00:00<?, ?it/s]

************************
Finished crawling data!


In [8]:
with open('ShopeeMallSentiment.csv', 'w',  newline = '') as file_output:
    
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')

Mission Completed!
